# Transformer all in one

In [149]:
%%capture
%load_ext autoreload
%autoreload 2

## Imports

In [150]:
import transformer_all
from transformer_all import Config, TrainHandler, DataHandler, Modules, Format

In [151]:
format = Format(style="solarized-light")
print_py = format.print_py
print_obj = format.print_obj
print_src = format.print_src

In [152]:
config=Config()
print_obj(config)

Config(seed=0,
       preview=slice(None, 10, None),
       example_item=1337,
       max_length=512,
       min_freq=5,
       default_token='<unk>',
       special_tokens=['<unk>', '<pad>'],
       batch_size=512,
       lr=0.001,
       num_epochs=20,
       heads=4,
       emb=32,
       device='cuda')



In [153]:
train_handler = TrainHandler(config=config)
print_src(train_handler.__init__)

    def __init__(self, config: Config):
        self.config = config
        np.random.seed(self.config.seed)
        torch.manual_seed(self.config.seed)
        torch.cuda.manual_seed(self.config.seed)
        torch.backends.cudnn.deterministic = True



In [154]:
data_handler = DataHandler(config=config)
print_src(data_handler.__init__)

    def __init__(self, config: Config):
        self.config = config
        self.dataset = datasets.load_dataset("zapsdcn/imdb", cache_dir="../data/classification/")
        self.train_dataset = self.dataset['train'] 
        self.validation_dataset = self.dataset['validation']
        self.test_dataset = self.dataset['test']
        self.tokenizer = torchtext.data.utils.get_tokenizer("basic_english")
        self.trim = slice(None,config.max_length,None)
        self.tokenize_all()
        self.set_vocab()
        self.numericalize_all()
        self.format_all()
        self.set_data_loaders()



In [157]:
train_handler.load(data_handler=data_handler)
print_src(train_handler.load)

    def load(self, data_handler: DataHandler):
        self.data_handler = data_handler
        self.model = Modules.Transformer(config=self.config, vocab_size=len(self.data_handler.vocab)).to(self.config.device)
        self.opt = torch.optim.Adam(lr=self.config.lr, params=self.model.parameters())



In [159]:
print_src(Modules.Transformer)

    class Transformer(torch.nn.Module):
        def __init__(self, config: Config, vocab_size: int):
            super().__init__()
            self.config = config
            self.token_embedding = torch.nn.Embedding(embedding_dim=self.config.emb, num_embeddings=vocab_size)
            self.pos_embedding = torch.nn.Embedding(embedding_dim=self.config.emb, num_embeddings=self.config.max_length)
            self.tblock = Modules.TransformerBlock(config=self.config)
            self.toprobs = torch.nn.Linear(self.config.emb, 2)

        def forward(self, x):
            tokens = self.token_embedding(x)
            b, t, e = tokens.shape
            positions = self.pos_embedding(torch.arange(t, device=self.config.device))[None, :, :].expand(b, t, e)
            x = tokens + positions
            x, W = self.tblock(x)
            x = torch.mean(x, dim=1)
            x = self.toprobs(x)
            return F.log_softmax(x, dim=1), W



In [ ]:
train_handler.train()

In [ ]:
train_handler.save()